라이브러리 확인

In [9]:
import seaborn as sns                                                                     # 데이터 시각화 라이브러리
import pandas as pd       
import numpy as np                                                                # 데이터 처리 라이브러리
from sklearn.model_selection import train_test_split, RandomizedSearchCV                  # 학습 데이터와 테스트 데이터로 나누는 라이브러리
from sklearn.impute import SimpleImputer                                                  # 결측치 처리 라이브러리                          
from sklearn.compose import ColumnTransformer                                             # 
from sklearn.pipeline import Pipeline                                                     # 파이프라인을 만들기 위한 라이브러리                     
from sklearn.tree import DecisionTreeRegressor                                            # 결정트리 회귀 모델 라이브러리       
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score       # 모델 평가 라이브러리
from sklearn.preprocessing import OneHotEncoder, StandardScaler                           # 원핫인코딩, 표준화 라이브러리

데이터

In [2]:
# 데이터 불러오기
submission = pd.read_csv('./house-prices/sample_submission.csv')
train = pd.read_csv('./house-prices/train.csv')
test = pd.read_csv('./house-prices/test.csv')

# 데이터 분리 
y = train['SalePrice']                      # SalePrice 컬럼을 y 데이터로 사용
train = train.drop('SalePrice', axis=1)     # SalePrice 컬럼을 삭제한 데이터를 train 데이터로 사용
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

탐색적 자료 분석 : 불 필요한 변수 제거 

In [7]:
numerical_cols = train.select_dtypes(include=['int64', 'float64']).columns      # 수치 데이터 컬럼만 추출
categorical_cols = train.select_dtypes(include=['object']).columns              # 범주형 데이터 컬럼만 추출

# 수치 데이터 전처리(결측치 처리, 표준화)
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),      # 결측치를 중앙값으로 채우기
    ('scaler', StandardScaler())                        # 표준화
])

# 범주형 데이터 전처리(결측치 처리, 원핫인코딩)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),   # 결측치를 최빈값으로 채우기
    ('onehot', OneHotEncoder(handle_unknown='ignore'))      # 원핫인코딩
])

# 전처리 파이프라인 만들기
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),       # 수치 데이터 파이프라인
        ('cat', categorical_transformer, categorical_cols)    # 범주형 데이터 파이프라인
    ])

모델 정의

In [10]:
model = DecisionTreeRegressor()   # 모델 생성

# 파이프라인 구축 
pipline = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', model)])    # 전처리 파이프라인과 모델을 연결

# 랜덤 서치 
# 훈련/ 테스트 데이터 분리 
X_train, X_valid, y_train, y_valid = train_test_split(train, y, test_size=0.2, random_state=0)   # 훈련 데이터와 테스트 데이터로 분리

# 랜덤 서치를 위한 하이퍼파라미터 설정
param_distributions = {
    # 'preprocessor__num__imputer__strategy': ['mean', 'median'],           # 수치 데이터 결측치 처리 전략
    'regressor__max_depth': np.concatenate((np.arange(3, 11), [None])),     # 결정트리의 최대 깊이
    'regressor__min_samples_split': np.arange(2, 11),                       # 분할되기 위해 노드가 가져야 하는 최소 샘플 수
    'regressor__min_samples_leaf': np.arange(1, 5)                          # 리프 노드가 가져야 하는 최소 샘플 수
}

random_search = RandomizedSearchCV(pipline, param_distributions, n_iter=10, cv=5, random_state=0)  # 랜덤 서치
random_search.fit(X_train, y_train)   # 랜덤 서치 훈련

print('최적 하이퍼파라미터: ', random_search.best_params_)   # 최적 하이퍼파라미터 출력

최적 하이퍼파라미터:  {'regressor__min_samples_split': 8, 'regressor__min_samples_leaf': 2, 'regressor__max_depth': 7}


모델 예측 및 평가 